In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import keras
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator


In [2]:
# Load data
train_data = pd.read_csv("E:/CodeClause/Artificial_Int/Blindness_Detection/train.csv")
test_data = pd.read_csv("E:/CodeClause/Artificial_Int/Blindness_Detection/test.csv")
train_images_path = "E:/CodeClause/Artificial_Int/Blindness_Detection/train_images/"
test_images_path = "E:/CodeClause/Artificial_Int/Blindness_Detection/test_images/"


In [3]:
# Data preprocessing
def preprocess_image(image):
    image = cv2.addWeighted(image, 4, cv2.GaussianBlur(image, (0, 0), 30), -4, 128)
    image = cv2.resize(image, (120, 120))
    return image

def load_and_preprocess_images(image_ids, image_path):
    images = []
    for image_id in image_ids:
        image = cv2.imread(os.path.join(image_path, image_id + ".png"))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        images.append(image)
    return np.array(images)



In [4]:
# Split data
trainX, testX, trainY, testY = train_test_split(train_data['id_code'], train_data['diagnosis'], test_size=0.2)

train_imgs = load_and_preprocess_images(trainX, train_images_path)
test_imgs = load_and_preprocess_images(testX, train_images_path)


In [5]:

# Model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), input_shape=(120, 120, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # 5 classes for diagnosis

model.compile(optimizer=Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [6]:
# Data augmentation
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

datagen.fit(train_imgs)



In [7]:
# Train the model
history = model.fit_generator(datagen.flow(train_imgs, trainY, batch_size=32),
                              steps_per_epoch=len(train_imgs) / 32,
                              validation_data=(test_imgs, testY),
                              epochs=15)



C:\Users\ABHIJEET\AppData\Local\Temp\ipykernel_5544\2320528045.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(train_imgs, trainY, batch_size=32),


Epoch 1/15
91/91 [==============================] - 44s 448ms/step - loss: 1.7690 - accuracy: 0.5814 - val_loss: 112.0973 - val_accuracy: 0.2483
Epoch 2/15
91/91 [==============================] - 41s 444ms/step - loss: 0.9913 - accuracy: 0.6497 - val_loss: 139.4274 - val_accuracy: 0.2483
Epoch 3/15
91/91 [==============================] - 41s 448ms/step - loss: 0.9664 - accuracy: 0.6623 - val_loss: 69.9030 - val_accuracy: 0.4884
Epoch 4/15
91/91 [==============================] - 41s 449ms/step - loss: 0.9332 - accuracy: 0.6753 - val_loss: 95.0644 - val_accuracy: 0.3029
Epoch 5/15
91/91 [==============================] - 43s 472ms/step - loss: 0.8757 - accuracy: 0.6965 - val_loss: 98.0522 - val_accuracy: 0.5089
Epoch 6/15
91/91 [==============================] - 43s 472ms/step - loss: 0.8746 - accuracy: 0.6934 - val_loss: 46.7876 - val_accuracy: 0.6003
Epoch 7/15
91/91 [==============================] - 43s 473ms/step - loss: 0.8346 - accuracy: 0.7047 - val_loss: 111.8095 - val_accura

In [8]:
# Evaluation
score = model.evaluate(test_imgs, testY, verbose=0)
print("Accuracy: %.2f%%" % (score[1] * 100))


Accuracy: 59.07%


In [9]:
# Prediction
test_imgs = load_and_preprocess_images(test_data['id_code'], test_images_path)

In [16]:
print("test_imgs shape:", test_imgs.shape)
print("predictions shape:", predictions.shape)

test_imgs shape: (1928, 120, 120, 3)
predictions shape: (1928, 5)


In [10]:
# Make predictions using the model
predictions = model.predict(test_imgs)

61/61 [==============================] - 5s 84ms/step


In [19]:
# Extract the predicted class labels (assuming a classification task)
flattened_predictions = predictions.argmax(axis=-1)

In [20]:
# Create a submission DataFrame
submission = pd.DataFrame({'id_code': test_data['id_code'], 'diagnosis': flattened_predictions})

# Save the DataFrame to a CSV file
submission.to_csv("submission.csv", index=False)
print("Submission file created.")

Submission file created.
